## FAO deforestation widget:

* Dynamic sentence and ranked list
* Data from Carto table

Original data source from [FAO FLUDE](http://www.fao.org/forest-resources-assessment/explore-data/flude/en/),
`FOREST AREA AND CHARACTERISTICS` table,	`Deforest` and `HumDef`	(Deforestation of which human induced), units in `ha/yr`.

In [1]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

In [2]:
# For FAO data, only country (admin0) level data are avaiable

adm0 = 'ARG'  

In [3]:
# First, get the FAO data from a carto table

sql = ("SELECT fao.country, fao.name, "
        "fao.deforest, fao.humdef, fao.year "
        "FROM table_1_forest_area_and_characteristics as fao "
        f"WHERE fao.country = '{adm0}'"
        )

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')
pprint(r.json())

try:
    fao_deforest = r.json().get('rows')
except:
    fao_deforest = None

https://wri-01.carto.com/api/v2/sql?q=SELECT+fao.country%2C+fao.name%2C+fao.deforest%2C+fao.humdef%2C+fao.year+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.country+%3D+%27ARG%27 

{'fields': {'country': {'type': 'string'},
            'deforest': {'type': 'number'},
            'humdef': {'type': 'number'},
            'name': {'type': 'string'},
            'year': {'type': 'number'}},
 'rows': [{'country': 'ARG',
           'deforest': None,
           'humdef': None,
           'name': 'Argentina',
           'year': 1990},
          {'country': 'ARG',
           'deforest': 235,
           'humdef': 235,
           'name': 'Argentina',
           'year': 2000},
          {'country': 'ARG',
           'deforest': 336,
           'humdef': 336,
           'name': 'Argentina',
           'year': 2005},
          {'country': 'ARG',
           'deforest': 301,
           'humdef': 301,
           'name': 'Argentina',
           'year': 2010},
          {'country': 'ARG'

In [4]:
for row in fao_deforest:
    print(row)

{'country': 'ARG', 'name': 'Argentina', 'deforest': None, 'humdef': None, 'year': 1990}
{'country': 'ARG', 'name': 'Argentina', 'deforest': 235, 'humdef': 235, 'year': 2000}
{'country': 'ARG', 'name': 'Argentina', 'deforest': 336, 'humdef': 336, 'year': 2005}
{'country': 'ARG', 'name': 'Argentina', 'deforest': 301, 'humdef': 301, 'year': 2010}
{'country': 'ARG', 'name': 'Argentina', 'deforest': None, 'humdef': None, 'year': 2015}
{'country': 'ARG', 'name': 'Argentina', 'deforest': None, 'humdef': None, 'year': 9999}


## Build a dynamic sentence

1. Identify years with valid data returned from the table (expose that to the user in the selector)
1. Create a dynamic sentence with `deforest ` data.
1. Extend the sentence if `humdef` exists (e.g. Argentina).


In [5]:
valid_years = []
for row in fao_deforest:
    if row.get('deforest') is not None:
        valid_years.append(row.get('year'))

# Here is a list of valid years that should go into the selector:
print(valid_years)

[2000, 2005, 2010]


In [6]:
def get_deforest(response, year):
    """Returns deforestation rate present in response"""
    for row in response:
        if row.get('year') == year:
            return row.get('deforest')

def get_humanDeforest(response,year):
    """Returns the human-induced deforestation rate present in the response"""
    for row in response:
        if row.get('year') == year:
            return row.get('humdef')

In [7]:
get_deforest(fao_deforest, 2000)

235

In [8]:
get_humanDeforest(fao_deforest, 2000)

235

In [9]:
selected_year =  valid_years[0]  #  e.g. of a selected year from the list
deforest = get_deforest(fao_deforest, selected_year)
human_deforest = get_humanDeforest(fao_deforest, selected_year)

ds = (f"From {selected_year}–{selected_year + 5}, the rate of deforestation in "
      f"{iso_to_countries[adm0]} was {deforest:,g} ha/yr")

if human_deforest:
    ds += f", of which {human_deforest:,g} ha/yr was due to human activity."
else:
    ds +="."

print(ds)

From 2000–2005, the rate of deforestation in Argentina was 235 ha/yr, of which 235 ha/yr was due to human activity.


## Ranked list

Retrieve an ordered list of countries by deforestation rate for a specified year. Show the ranked list ±2 elements as with other ranked lists.

In [10]:
selected_year =  valid_years[0]  #  e.g. of a selected year from the list

sql = (
        "WITH mytable AS (SELECT fao.country as iso, "
        "fao.name,fao.deforest, fao.humdef "
        "FROM table_1_forest_area_and_characteristics as fao "
        f"WHERE fao.year = {selected_year} "
        "AND deforest is not null), "
        "rank AS ( "
        "SELECT deforest, iso "
        "from mytable "
        "ORDER BY mytable.deforest DESC) "
        "SELECT row_number() over () as rank, iso, deforest from rank "
)

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')

rank = r.json().get('rows')

https://wri-01.carto.com/api/v2/sql?q=WITH+mytable+AS+%28SELECT+fao.country+as+iso%2C+fao.name%2Cfao.deforest%2C+fao.humdef+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.year+%3D+2000+AND+deforest+is+not+null%29%2C+rank+AS+%28+SELECT+deforest%2C+iso+from+mytable+ORDER+BY+mytable.deforest+DESC%29+SELECT+row_number%28%29+over+%28%29+as+rank%2C+iso%2C+deforest+from+rank+ 



In [11]:
for row in rank:
    print(row)

{'rank': 1, 'iso': 'BRA', 'deforest': 3277}
{'rank': 2, 'iso': 'TZA', 'deforest': 400}
{'rank': 3, 'iso': 'AUS', 'deforest': 326}
{'rank': 4, 'iso': 'COL', 'deforest': 314.99}
{'rank': 5, 'iso': 'VEN', 'deforest': 288}
{'rank': 6, 'iso': 'BOL', 'deforest': 270}
{'rank': 7, 'iso': 'ARG', 'deforest': 235}
{'rank': 8, 'iso': 'SDN', 'deforest': 174.415}
{'rank': 9, 'iso': 'PER', 'deforest': 143}
{'rank': 10, 'iso': 'KEN', 'deforest': 112.4}
{'rank': 11, 'iso': 'MLI', 'deforest': 109}
{'rank': 12, 'iso': 'ECU', 'deforest': 92.787}
{'rank': 13, 'iso': 'GTM', 'deforest': 73}
{'rank': 14, 'iso': 'CAN', 'deforest': 48}
{'rank': 15, 'iso': 'PHL', 'deforest': 47.19}
{'rank': 16, 'iso': 'LAO', 'deforest': 35.3}
{'rank': 17, 'iso': 'SPM', 'deforest': 20}
{'rank': 18, 'iso': 'DZA', 'deforest': 18}
{'rank': 19, 'iso': 'BRN', 'deforest': 16}
{'rank': 20, 'iso': 'MRT', 'deforest': 9.8}
{'rank': 21, 'iso': 'FIN', 'deforest': 8}
{'rank': 22, 'iso': 'KOR', 'deforest': 6}
{'rank': 23, 'iso': 'NOR', 'defore

### extend dynamic sentence

The dynamic sentence should also end with the following statement about country rank:

In [12]:
def get_countryRank(response, iso):
    for row in response:
        if row.get('iso') == iso:
            return row.get('rank')

In [13]:
iso_rank = get_countryRank(rank, adm0)

In [14]:
ds += (f" In relation to other countries this was the {iso_rank}th largest change.")
ds

'From 2000–2005, the rate of deforestation in Argentina was 235 ha/yr, of which 235 ha/yr was due to human activity. In relation to other countries this was the 7th largest change.'